In [1]:
import keras

Using TensorFlow backend.


In [2]:
print('keras: ', keras.__version__)

keras:  2.0.8


In [3]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

In [4]:
# Get images
image_raw = img_to_array(load_img('green.jpg'))
image_raw = np.array(image_raw, dtype=float)
#Load weights
inception = InceptionResNetV2(weights='imagenet', include_top=True)

In [5]:
X = rgb2lab(1.0/255*image_raw)[:,:,0]
Y = rgb2lab(1.0/255*image_raw)[:,:,1:]
Y /= 128
X = X.reshape(1, 256, 256, 1)
Y = Y.reshape(1, 256, 256, 2)

In [6]:
def conv_stack(data, filters, s):
        output = Conv2D(filters, (3, 3), strides=s, activation='relu', padding='same')(data)
        output = BatchNormalization()(output)
        return output

#Create inception embedding
img_path = 'dog.jpg'
img = image.load_img(img_path, target_size=(299, 299))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)
embed = inception.predict(img)
embed_input = Input(shape=(1000,))

#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = conv_stack(encoder_input, 64, 2)
encoder_output = conv_stack(encoder_output, 128, 2)
encoder_output = conv_stack(encoder_output, 256, 2)
encoder_output = conv_stack(encoder_output, 512, 1)
encoder_output = conv_stack(encoder_output, 256, 1)

#Fusion
# y_mid: (None, 256, 28, 28)
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Permute((2, 1))(fusion_output) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([fusion_output, encoder_output], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu')(fusion_output) 

#Decoder
decoder_output = UpSampling2D((2, 2))(fusion_output)
decoder_output = conv_stack(decoder_output, 128, 1)
decoder_output = conv_stack(decoder_output, 64, 1)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = conv_stack(decoder_output, 32, 1)
decoder_output = conv_stack(decoder_output, 16, 1)
decoder_output = Conv2D(2, (2, 2), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

In [7]:
# Finish model
model.compile(optimizer='rmsprop', loss='mse')
model.fit([X, embed], Y,
    batch_size=1,
    epochs=1000)

Epoch 1/1000
1/1 [==============================] - 7s - loss: 0.6054
Epoch 2/1000
1/1 [==============================] - 0s - loss: 0.4840
Epoch 3/1000
1/1 [==============================] - 0s - loss: 0.3169
Epoch 4/1000
1/1 [==============================] - 0s - loss: 0.1944
Epoch 5/1000
1/1 [==============================] - 0s - loss: 0.2097
Epoch 6/1000
1/1 [==============================] - 0s - loss: 0.1443
Epoch 7/1000
1/1 [==============================] - 0s - loss: 0.1301
Epoch 8/1000
1/1 [==============================] - 0s - loss: 0.1208
Epoch 9/1000
1/1 [==============================] - 0s - loss: 0.1049
Epoch 10/1000
1/1 [==============================] - 0s - loss: 0.0985
Epoch 11/1000
1/1 [==============================] - 0s - loss: 0.1053
Epoch 12/1000
1/1 [==============================] - 0s - loss: 0.0984
Epoch 13/1000
1/1 [==============================] - 0s - loss: 0.0873
Epoch 14/1000
1/1 [==============================] - 0s - loss: 0.0875
Epoch 15/1000
1

1/1 [==============================] - 0s - loss: 0.0065
Epoch 117/1000
1/1 [==============================] - 0s - loss: 0.0058
Epoch 118/1000
1/1 [==============================] - 1s - loss: 0.0063
Epoch 119/1000
1/1 [==============================] - 1s - loss: 0.0056
Epoch 120/1000
1/1 [==============================] - 1s - loss: 0.0064
Epoch 121/1000
1/1 [==============================] - 0s - loss: 0.0063
Epoch 122/1000
1/1 [==============================] - 0s - loss: 0.0076
Epoch 123/1000
1/1 [==============================] - 0s - loss: 0.0071
Epoch 124/1000
1/1 [==============================] - 0s - loss: 0.0055
Epoch 125/1000
1/1 [==============================] - 0s - loss: 0.0091
Epoch 126/1000
1/1 [==============================] - 0s - loss: 0.0053
Epoch 127/1000
1/1 [==============================] - 0s - loss: 0.0041
Epoch 128/1000
1/1 [==============================] - 0s - loss: 0.0034
Epoch 129/1000
1/1 [==============================] - 0s - loss: 0.0039
Epoch 1

1/1 [==============================] - 1s - loss: 0.0023
Epoch 231/1000
1/1 [==============================] - 1s - loss: 0.0021
Epoch 232/1000
1/1 [==============================] - 1s - loss: 0.0023
Epoch 233/1000
1/1 [==============================] - 1s - loss: 0.0031
Epoch 234/1000
1/1 [==============================] - 1s - loss: 0.0033
Epoch 235/1000
1/1 [==============================] - 1s - loss: 0.0021
Epoch 236/1000
1/1 [==============================] - 1s - loss: 0.0020
Epoch 237/1000
1/1 [==============================] - 1s - loss: 0.0027
Epoch 238/1000
1/1 [==============================] - 1s - loss: 0.0027
Epoch 239/1000
1/1 [==============================] - 1s - loss: 0.0039
Epoch 240/1000
1/1 [==============================] - 1s - loss: 0.0024
Epoch 241/1000
1/1 [==============================] - 1s - loss: 0.0023
Epoch 242/1000
1/1 [==============================] - 1s - loss: 0.0030
Epoch 243/1000
1/1 [==============================] - 1s - loss: 0.0029
Epoch 2

1/1 [==============================] - 1s - loss: 0.0017
Epoch 345/1000
1/1 [==============================] - 1s - loss: 0.0018
Epoch 346/1000
1/1 [==============================] - 1s - loss: 0.0019
Epoch 347/1000
1/1 [==============================] - 1s - loss: 0.0026
Epoch 348/1000
1/1 [==============================] - 1s - loss: 0.0021
Epoch 349/1000
1/1 [==============================] - 1s - loss: 0.0018
Epoch 350/1000
1/1 [==============================] - 1s - loss: 0.0019
Epoch 351/1000
1/1 [==============================] - 0s - loss: 0.0021
Epoch 352/1000
1/1 [==============================] - 0s - loss: 0.0028
Epoch 353/1000
1/1 [==============================] - 0s - loss: 0.0022
Epoch 354/1000
1/1 [==============================] - 0s - loss: 0.0016
Epoch 355/1000
1/1 [==============================] - 0s - loss: 0.0016
Epoch 356/1000
1/1 [==============================] - 0s - loss: 0.0018
Epoch 357/1000
1/1 [==============================] - 0s - loss: 0.0022
Epoch 3

1/1 [==============================] - 1s - loss: 0.0014
Epoch 459/1000
1/1 [==============================] - 1s - loss: 0.0017
Epoch 460/1000
1/1 [==============================] - 1s - loss: 0.0018
Epoch 461/1000
1/1 [==============================] - 1s - loss: 0.0019
Epoch 462/1000
1/1 [==============================] - 1s - loss: 0.0016
Epoch 463/1000
1/1 [==============================] - 1s - loss: 0.0018
Epoch 464/1000
1/1 [==============================] - 1s - loss: 0.0014
Epoch 465/1000
1/1 [==============================] - 1s - loss: 0.0015
Epoch 466/1000
1/1 [==============================] - 1s - loss: 0.0021
Epoch 467/1000
1/1 [==============================] - 1s - loss: 0.0018
Epoch 468/1000
1/1 [==============================] - 1s - loss: 0.0014
Epoch 469/1000
1/1 [==============================] - 0s - loss: 0.0015
Epoch 470/1000
1/1 [==============================] - 0s - loss: 0.0013
Epoch 471/1000
1/1 [==============================] - 0s - loss: 0.0015
Epoch 4

1/1 [==============================] - 1s - loss: 0.0013
Epoch 573/1000
1/1 [==============================] - 1s - loss: 0.0013
Epoch 574/1000
1/1 [==============================] - 1s - loss: 0.0014
Epoch 575/1000
1/1 [==============================] - 1s - loss: 0.0016
Epoch 576/1000
1/1 [==============================] - 1s - loss: 0.0014
Epoch 577/1000
1/1 [==============================] - 1s - loss: 0.0013
Epoch 578/1000
1/1 [==============================] - 1s - loss: 0.0014
Epoch 579/1000
1/1 [==============================] - 1s - loss: 0.0015
Epoch 580/1000
1/1 [==============================] - 1s - loss: 0.0015
Epoch 581/1000
1/1 [==============================] - 1s - loss: 0.0016
Epoch 582/1000
1/1 [==============================] - 1s - loss: 0.0014
Epoch 583/1000
1/1 [==============================] - 1s - loss: 0.0013
Epoch 584/1000
1/1 [==============================] - 1s - loss: 0.0015
Epoch 585/1000
1/1 [==============================] - 1s - loss: 0.0015
Epoch 5

1/1 [==============================] - 0s - loss: 0.0015
Epoch 687/1000
1/1 [==============================] - 0s - loss: 0.0012
Epoch 688/1000
1/1 [==============================] - 0s - loss: 0.0011
Epoch 689/1000
1/1 [==============================] - 0s - loss: 0.0013
Epoch 690/1000
1/1 [==============================] - 0s - loss: 0.0015
Epoch 691/1000
1/1 [==============================] - 0s - loss: 0.0012
Epoch 692/1000
1/1 [==============================] - 0s - loss: 0.0012
Epoch 693/1000
1/1 [==============================] - 0s - loss: 0.0015
Epoch 694/1000
1/1 [==============================] - 0s - loss: 0.0014
Epoch 695/1000
1/1 [==============================] - 0s - loss: 0.0011
Epoch 696/1000
1/1 [==============================] - 1s - loss: 0.0011
Epoch 697/1000
1/1 [==============================] - 1s - loss: 0.0015
Epoch 698/1000
1/1 [==============================] - 1s - loss: 0.0012
Epoch 699/1000
1/1 [==============================] - 1s - loss: 0.0010
Epoch 7

1/1 [==============================] - 1s - loss: 0.0013
Epoch 800/1000
1/1 [==============================] - 0s - loss: 0.0011
Epoch 801/1000
1/1 [==============================] - 0s - loss: 0.0010
Epoch 802/1000
1/1 [==============================] - 0s - loss: 0.0013
Epoch 803/1000
1/1 [==============================] - 0s - loss: 0.0012
Epoch 804/1000
1/1 [==============================] - 0s - loss: 0.0014
Epoch 805/1000
1/1 [==============================] - 0s - loss: 0.0011
Epoch 806/1000
1/1 [==============================] - 0s - loss: 0.0010
Epoch 807/1000
1/1 [==============================] - 0s - loss: 0.0011
Epoch 808/1000
1/1 [==============================] - 0s - loss: 0.0012
Epoch 809/1000
1/1 [==============================] - 0s - loss: 0.0012
Epoch 810/1000
1/1 [==============================] - 0s - loss: 0.0013
Epoch 811/1000
1/1 [==============================] - 1s - loss: 0.0015
Epoch 812/1000
1/1 [==============================] - 1s - loss: 0.0011
Epoch 8

1/1 [==============================] - 0s - loss: 0.0011
Epoch 913/1000
1/1 [==============================] - 0s - loss: 0.0011
Epoch 914/1000
1/1 [==============================] - 1s - loss: 9.0622e-04
Epoch 915/1000
1/1 [==============================] - 1s - loss: 0.0012
Epoch 916/1000
1/1 [==============================] - 1s - loss: 0.0012
Epoch 917/1000
1/1 [==============================] - 1s - loss: 0.0013
Epoch 918/1000
1/1 [==============================] - 0s - loss: 9.2741e-04
Epoch 919/1000
1/1 [==============================] - 0s - loss: 0.0010
Epoch 920/1000
1/1 [==============================] - 0s - loss: 0.0013
Epoch 921/1000
1/1 [==============================] - 0s - loss: 0.0014
Epoch 922/1000
1/1 [==============================] - 0s - loss: 0.0013
Epoch 923/1000
1/1 [==============================] - 0s - loss: 0.0011
Epoch 924/1000
1/1 [==============================] - 0s - loss: 9.8415e-04
Epoch 925/1000
1/1 [==============================] - 0s - loss: 0.

In [8]:
output = model.predict([X, embed])
output *= 128
# Output colorizations
cur = np.zeros((256, 256, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]
imsave("img_result.png", lab2rgb(cur))
imsave("img_gray_version.png", rgb2gray(lab2rgb(cur)))

/Users/ewallner/.brew/lib/python3.6/site-packages/skimage/color/colorconv.py:985: UserWarning: Color data out of range: Z < 0 in 71 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
/Users/ewallner/.brew/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/Users/ewallner/.brew/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
